In [ ]:
%%capture

import importlib

from compare_tools import *
import matplotlib.pyplot as plt
import numpy as np
import cv2
from cnn import cnn_predict_grid
from skimage import measure
from tools.load_nc import load_nc_cmems, load_nc_sinmod
from matplotlib.patches import Rectangle
import pandas as pd
from scipy.interpolate import griddata, interp2d
from scipy.signal import resample
import tools.grid_conversion
importlib.reload(tools.grid_conversion)
from scipy.spatial import cKDTree

# Prepare data for analysis

### Load NC data

In [ ]:
# Load satellite netcdf data
fPath = 'D:/master/data/cmems_data/global_10km/2016/full/phys_noland_2016_001.nc'
lons_sat,lats_sat,sst_sat,ssl_sat,uvel_sat,vvel_sat =  load_nc_cmems(fPath)
# Get SINMOD data
fPath = 'D:/master/data/cmems_data/sinmod/samples_2017.04.27_nonoverlap.nc'
x_sinmod,y_sinmod,lons_sinmod,lats_sinmod,sst_sinmod,ssl_sinmod,uvel_sinmod,vvel_sinmod =  tools.load_nc.load_nc_sinmod(fPath)

### Convert satellite data to north-polar sterographic projection

In [ ]:
# Get a polar steregraphic projection with cartesian irregular curvilinear grid for satellite data 
x_sat_2d, y_sat_2d = tools.grid_conversion.bl2xy(*np.meshgrid(lons_sat, lats_sat), FE=3254000, FN=2560000, xy_res=1, slon=58, SP=60)

### Interpolate satellite data to high enough resolution such that extracting closest neighbor from the satellite measurements based on queries from the selected SINMOD grid won't have duplicates.

In [ ]:
# Interpolate data and the curvilienar grid axes such that it has a higher resolution than SINMOD
sst_sat_it = interp2d_masked(sst_sat, lons_sat, lats_sat, 4, 'linear')
ssl_sat_it = interp2d_masked(ssl_sat, lons_sat, lats_sat, 4, 'linear')
uvel_sat_it = interp2d_masked(uvel_sat, lons_sat, lats_sat, 4, 'linear')
vvel_sat_it = interp2d_masked(vvel_sat, lons_sat, lats_sat, 4, 'linear')
x_sat_it = interp2d_masked(x_sat_2d.T, lons_sat, lats_sat, 4, 'linear')
y_sat_it = interp2d_masked(y_sat_2d.T, lons_sat, lats_sat, 4, 'linear')

### Select a grid that will be our basis of analysis

In [ ]:
# Get SINMOD Grid
x_bnds = [720000, 1560000]
y_bnds = [650000, 1200000]
x_idxs = np.where((x_sinmod >= x_bnds[0]) & (x_sinmod <= x_bnds[1]))[0]
y_idxs = np.where((y_sinmod >= y_bnds[0]) & (y_sinmod <= y_bnds[1]))[0]

x_sinmod = np.ma.array(x_sinmod[x_idxs])
y_sinmod = np.ma.array(y_sinmod[y_idxs])
lons_sinmod = np.ma.array(lons_sinmod[x_idxs[0]:x_idxs[-1]+1, y_idxs[0]:y_idxs[-1]+1])
lats_sinmod = np.ma.array(lats_sinmod[x_idxs[0]:x_idxs[-1]+1, y_idxs[0]:y_idxs[-1]+1])
sst_sinmod = np.ma.array(sst_sinmod[x_idxs[0]:x_idxs[-1]+1, y_idxs[0]:y_idxs[-1]+1])
ssl_sinmod = np.ma.array(ssl_sinmod[x_idxs[0]:x_idxs[-1]+1, y_idxs[0]:y_idxs[-1]+1])
uvel_sinmod = np.ma.array(uvel_sinmod[x_idxs[0]:x_idxs[-1]+1, y_idxs[0]:y_idxs[-1]+1])
vvel_sinmod = np.ma.array(vvel_sinmod[x_idxs[0]:x_idxs[-1]+1, y_idxs[0]:y_idxs[-1]+1])

### Create k-d tree to create a satellite grid with same axes as the selected SINMOD grid

In [ ]:
# Create cKDTree object to represent the full flattened cartesian coordinates of the satellite data grid
tree = cKDTree(list(zip(x_sat_it.flatten(), y_sat_it.flatten())))

In [ ]:
# Find indices of the satellite data nearest to the SINMOD coordinates we use
x_sinmod_2d, y_sinmod_2d = np.meshgrid(x_sinmod, y_sinmod)
d, inds = tree.query(list(zip(x_sinmod_2d.ravel(), y_sinmod_2d.ravel())), k = 1)

# Extract satellite data by query indexes to get same grid axes and apply the same landmass mask
sinmod_mask = np.ma.getmask(ssl_sinmod)
sst_sat_samegrid = np.ma.masked_where(sinmod_mask, sst_sat_it.flatten()[inds].reshape(uvel_sinmod.T.shape).T)
ssl_sat_samegrid = np.ma.masked_where(sinmod_mask, ssl_sat_it.flatten()[inds].reshape(uvel_sinmod.T.shape).T)
uvel_sat_samegrid = np.ma.masked_where(sinmod_mask, uvel_sat_it.flatten()[inds].reshape(uvel_sinmod.T.shape).T)
vvel_sat_samegrid = np.ma.masked_where(sinmod_mask, vvel_sat_it.flatten()[inds].reshape(uvel_sinmod.T.shape).T)

In [ ]:
# Rotate velocities to fit the shifted pole stereographic projection
uvel_sinmod, vvel_sinmod = rotate_vector(uvel_sinmod, vvel_sinmod, lons_sinmod, 58)
uvel_sat_samegrid, vvel_sat_samegrid = rotate_vector(uvel_sat_samegrid, vvel_sat_samegrid, lons_sinmod, 58)

# Process satellite data

### Perform pyramid sliding window on grid and return use CNN model to predict rectangles

In [ ]:
%%capture
model_fpath = 'D:/master/models/2016/cnn_mult_velocities_9652.h5'
cnn_win_sizes = [((int(9), int(5)), 2, 2),((int(12), int(7)), 3, 2),((int(15), int(9)), 3, 3)]
cnn_data = [lons_sinmod, lats_sinmod, x_sinmod, y_sinmod, ssl_sat_samegrid, uvel_sat_samegrid, vvel_sat_samegrid]
# Get the cyclone and anti-cyclone rectangle
cyclones_sat, anticyclones_sat = cnn_predict_grid(cnn_data, cnn_win_sizes, 0.95, model_fpath, storedir="sat1")

In [ ]:
# Group all predicted boxes according to overlap and number of rectangles in a group
cyc_sat, _ = cv2.groupRectangles(rectList=cyclones_sat, groupThreshold=3, eps=0.1)
acyc_sat, _ = cv2.groupRectangles(rectList=anticyclones_sat, groupThreshold=3, eps=0.1)
nCyc, nAcyc = len(cyc_sat), len(acyc_sat)

### Use labeled binary clusters for each cyclone or anti-cyclone based on Okubo Weiss parameter and vorticity to find appropriate box surrounding the eddy

In [ ]:
import compare_tools
importlib.reload(compare_tools)

# Create mask for OW values above threshold, one that contains cyclones (negative vorticity) 
# and vice versa for anti-cyclones
# Default Okubo Weiss value was -1, want to use -8 to be absolutely sure, we have a more loose threshold to
# indicate cells with vortex characteristics
OW_start = -0.05
OW,vorticity,OW_mask,OW_cyc_mask,OW_acyc_mask = compare_tools.calc_OW(lons_sinmod,lats_sinmod,uvel_sat_samegrid,vvel_sat_samegrid,OW_start)

# We label all unique mask clusters
OW_mask_labeled = measure.label(OW_mask)
cyc_mask_labeled = measure.label(OW_cyc_mask)
acyc_mask_labeled = measure.label(OW_acyc_mask)

# Make the predicted boxes encompass the full cyclone or anti-cyclone clusters and return info about the eddy
cyc_ctrIdxs, cyc_nCells, cyc_minOW, cyc_new_sat = investigate_cluster(cyc_sat, OW, cyc_mask_labeled, 'cyclone')
acyc_ctrIdxs, acyc_nCells, acyc_minOW, acyc_new_sat = investigate_cluster(acyc_sat, OW, acyc_mask_labeled, 'anti-cyclone')

# Eddy boxes given in lat-lon coordiantes
cyc_boxcoords_sat = box2coords(x_sinmod, y_sinmod, cyc_new_sat)
acyc_boxcoords_sat = box2coords(x_sinmod, y_sinmod, acyc_new_sat)

### Show box surrounding the marked OW clusters before and after

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,12))
ax.imshow(OW_mask_labeled.T, cmap='nipy_spectral', origin='lower')
# Before
plot_eddies(fig, ax, cyc_sat, 'r') 
plot_eddies(fig, ax, acyc_sat, 'b') 
# After
plot_eddies(fig, ax, cyc_new_sat, 'r', numbered=False) 
plot_eddies(fig, ax, acyc_new_sat, 'b', numbered=False) 

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,12))
ax.imshow(vorticity.T, cmap='nipy_spectral', origin='lower')
# Before
plot_eddies(fig, ax, cyc_sat, 'r') 
plot_eddies(fig, ax, acyc_sat, 'b') 
# After
plot_eddies(fig, ax, cyc_new_sat, 'r', numbered=False) 
plot_eddies(fig, ax, acyc_new_sat, 'b', numbered=False) 

### Show plot of velocity streamplot and sea surface level with new boxes

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,12))#, dpi=500)
ax.pcolormesh(x_sinmod, y_sinmod, ssl_sat_samegrid.T, cmap='rainbow')
#ax.streamplot(X, Y, uvel, vvel.T, density=12)
#x.quiver(x_sinmod,y_sinmod,uvel_sat_samegrid.T,vvel_sat_samegrid.T,scale=12)
ax.streamplot(x_sinmod, y_sinmod, uvel_sat_samegrid.T, vvel_sat_samegrid.T, density=10) 
plot_eddies(fig, ax, cyc_boxcoords_sat, 'r') 
plot_eddies(fig, ax, acyc_boxcoords_sat, 'b') 

In [ ]:
# Eddy census to hold information about each eddy
cyclone_census = np.zeros((nCyc, 6))
anticyclone_census = np.zeros((nAcyc, 6))
# Eddy census dataframe to show eddy info table
cyc_census_df_sat = set_eddy_census(OW, anticyclone_census, acyc_ctrIdxs, acyc_nCells, acyc_new_sat, acyc_minOW, lons_sinmod, lats_sinmod, x_sinmod, y_sinmod, meastype='SINMOD')
acyc_census_df_sat = set_eddy_census(OW, anticyclone_census, acyc_ctrIdxs, acyc_nCells, acyc_new_sat, acyc_minOW, lons_sinmod, lats_sinmod, x_sinmod, y_sinmod, meastype='SINMOD')

# Process SINMOD data

### Perform pyramid sliding window on grid and return use CNN model to predict rectangles

In [ ]:
%%capture
model_fpath = 'D:/master/models/2016/cnn_mult_velocities_9652.h5'
cnn_win_sizes = [((int(9), int(5)), 2, 2),((int(12), int(7)), 3, 2),((int(15), int(9)), 3, 3)]
cnn_data = [lons_sinmod, lats_sinmod, x_sinmod, y_sinmod, ssl_sinmod, uvel_sinmod, vvel_sinmod]
# Get the cyclone and anti-cyclone rectangle
cyclones_sinmod, anticyclones_sinmod = cnn_predict_grid(cnn_data, cnn_win_sizes, 0.90, model_fpath, storedir="sinmod1")

In [ ]:
# Group all predicted boxes according to overlap and number of rectangles in a group
cyc_sinmod, _ = cv2.groupRectangles(rectList=cyclones_sinmod, groupThreshold=2, eps=0.15)
acyc_sinmod, _ = cv2.groupRectangles(rectList=anticyclones_sinmod, groupThreshold=2, eps=0.15)
nCyc, nAcyc = len(cyc_sinmod), len(acyc_sinmod)

### Use labeled binary clusters for each cyclone or anti-cyclone based on Okubo Weiss parameter and vorticity to find appropriate box surrounding the eddy

In [ ]:
%%capture
# Create mask for OW values above threshold, one that contains cyclones (negative vorticity) 
# and vice versa for anti-cyclones
# Default Okubo Weiss value was -1, want to use -8 to be absolutely sure, we have a more loose threshold to
# indicate cells with vortex characteristics
OW_start = -0.3
OW,vorticity,OW_mask,OW_cyc_mask,OW_acyc_mask = calc_OW(lons_sinmod,lats_sinmod,uvel_sinmod,vvel_sinmod,OW_start)

# We label all unique mask clusters
OW_mask_labeled = measure.label(OW_mask)
cyc_mask_labeled = measure.label(OW_cyc_mask)
acyc_mask_labeled = measure.label(OW_acyc_mask)

# Make the predicted boxes encompass the full cyclone or anti-cyclone clusters and return info about the eddy
cyc_ctrIdxs, cyc_nCells, cyc_minOW, cyc_new_sinmod = investigate_cluster(cyc_sinmod, OW, cyc_mask_labeled, 'cyclone')
acyc_ctrIdxs, acyc_nCells, acyc_minOW, acyc_new_sinmod = investigate_cluster(acyc_sinmod, OW, acyc_mask_labeled, 'anti-cyclone')

# Eddy boxes given in lat-lon coordiantes
cyc_boxcoords_sinmod = box2coords(x_sinmod, y_sinmod, cyc_new_sinmod)
acyc_boxcoords_sinmod = box2coords(x_sinmod, y_sinmod, acyc_new_sinmod)

### Show box surrounding the marked OW clusters before and after

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,12))
ax.imshow(OW_mask_labeled.T, cmap='nipy_spectral', origin='lower')
# Before
plot_eddies(fig, ax, cyc_sinmod, 'r') 
plot_eddies(fig, ax, acyc_sinmod, 'b') 
# After
plot_eddies(fig, ax, cyc_new_sinmod, 'r', numbered=False) 
plot_eddies(fig, ax, acyc_new_sinmod, 'b', numbered=False) 

### Show plot of velocity streamplot and sea surface level with new boxes

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,12))#, dpi=500)
ax.pcolormesh(x_sinmod, y_sinmod, ssl_sinmod.T, cmap='rainbow')
#ax.streamplot(X, Y, uvel, vvel.T, density=12)
#ax.quiver(x_sinmod,y_sinmod,uvel_sinmod.T,vvel_sinmod.T,scale=6)
ax.streamplot(x_sinmod, y_sinmod, uvel_sinmod.T, vvel_sinmod.T, density=10) 
plot_eddies(fig, ax, cyc_boxcoords_sinmod, 'r') 
plot_eddies(fig, ax, acyc_boxcoords_sinmod, 'b') 

In [ ]:
# Eddy census to hold information about each eddy
cyclone_census = np.zeros((nCyc, 6))
anticyclone_census = np.zeros((nAcyc, 6))
# Eddy census dataframe to show eddy info table
cyc_census_df_sinmod = set_eddy_census(OW, cyclone_census, cyc_ctrIdxs, cyc_nCells, cyc_new_sinmod, cyc_minOW, lons_sinmod, lats_sinmod, x_sinmod, y_sinmod, meastype='SINMOD')
acyc_census_df_sinmod = set_eddy_census(OW, anticyclone_census, acyc_ctrIdxs, acyc_nCells, acyc_new_sinmod, acyc_minOW, lons_sinmod, lats_sinmod, x_sinmod, y_sinmod, meastype='SINMOD')

# Compare the two

### Eddy census

In [ ]:
cyc_census_df_sat

In [ ]:
acyc_census_df_sat

In [ ]:
cyc_census_df_sinmod

In [ ]:
acyc_census_df_sinmod

### Compare data; temperature, sea surface level, streamplot

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(18,18))

def plot_eddies_wrapper(fig, ax, cyc, acyc):
    plot_eddies(fig, ax, cyc, 'r') 
    plot_eddies(fig, ax, acyc, 'b') 

axes[0,0].contourf(x_sinmod, y_sinmod, sst_sat_samegrid.T, levels=40, cmap='rainbow', vmin=6.5, vmax=10.5)
plot_eddies_wrapper(fig, axes[0,0], cyc_boxcoords_sat, acyc_boxcoords_sat)
axes[0,0].set_title('SST Satelite')

axes[0,1].contourf(x_sinmod, y_sinmod, sst_sinmod.T, levels=40, cmap='rainbow')
plot_eddies_wrapper(fig, axes[0,1], cyc_boxcoords_sinmod, acyc_boxcoords_sinmod)
axes[0,1].set_title('SST SINMOD')

axes[1,0].contourf(x_sinmod, y_sinmod, ssl_sat_samegrid.T, levels=40, cmap='rainbow')
plot_eddies_wrapper(fig, axes[1,0], cyc_boxcoords_sat, acyc_boxcoords_sat)
axes[1,0].set_title('SSL Satelite')

axes[1,1].contourf(x_sinmod, y_sinmod, ssl_sinmod.T, levels=40, cmap='rainbow')
plot_eddies_wrapper(fig, axes[1,1], cyc_boxcoords_sinmod, acyc_boxcoords_sinmod)
axes[1,1].set_title('SSL SINMOD')
